# Stage 14: Deployment & Monitoring

## 1) Reflection

### Deployment Risks for AAPL Stock Prediction Model

**Critical Failure Modes:**
- **Data Schema Drift**: Stock data format changes could break feature engineering pipeline
- **Market Regime Changes**: Model trained on 2023 data may underperform in different market conditions (bear/bull markets)
- **Feature Degradation**: Technical indicators lose predictive power during high volatility periods
- **API Latency Spikes**: Prediction delays during market hours could impact trading decisions
- **Label Delay**: Late arrival of actual prices prevents timely model validation

### Four-Layer Monitoring Strategy

**Data Layer:**
- Freshness: `data_age_minutes < 30` during market hours with PagerDuty alerts
- Quality: `null_rate < 5%` for core features (OHLCV), `schema_hash` validation
- Distribution: `price_range_p99 < $50` to catch extreme outliers

**Model Layer:**
- Performance: `rolling_7day_mae < 0.05` (5% normalized error threshold)
- Drift: `population_stability_index < 0.25` for key features triggers retraining
- Confidence: `prediction_variance < 0.1` ensures stable outputs

**System Layer:**
- Latency: `p95_response_time < 200ms` for prediction endpoint
- Reliability: `error_rate < 1%`, `cpu_utilization < 80%`
- Throughput: `requests_per_second > 10` during market hours

**Business Layer:**
- Accuracy: `directional_accuracy > 55%` (better than random prediction)
- Adoption: `daily_api_calls > 100` indicating user engagement
- Risk: `max_prediction_deviation < 10%` from historical norms

### Ownership & Handoffs

**Primary Owners:**
- Platform Team: System metrics, infrastructure alerts, 15-minute response SLA
- Data Science Team: Model performance, drift detection, weekly retraining
- Product Team: Business KPIs, user adoption, ROI tracking

**Alert Escalation:**
P0 (system failures) → Platform on-call → Data Science → Product Manager (15min → 1hr → 4hr escalation)

**Retraining Triggers:**
- Scheduled: Weekly with 90-day rolling window
- Performance: MAE > 0.08 or directional accuracy < 50%
- Drift: PSI > 0.25 for 3 consecutive days

## 2) Dashboard Sketch

### AAPL Model Monitoring Dashboard (4x3 Grid Layout)

**Top Row - Real-time System Health:**
- **API Performance Panel**: P95 latency gauge (target <200ms), request rate line chart, status indicators
- **Error Tracking Panel**: Error rate display, 4xx/5xx breakdown bar chart, recent error feed
- **Resource Utilization Panel**: CPU/Memory dual gauge, pod-level metrics, hourly trends

**Middle Row - Model Performance:**
- **Prediction Accuracy Panel**: Large MAE display, 30-day trend with confidence bands, 0.05 threshold line
- **Feature Drift Panel**: PSI heatmap for all features, color-coded (Green <0.1, Yellow 0.1-0.25, Red >0.25)
- **Model Predictions Panel**: Today's prediction histogram, prediction mean/variance time series

**Bottom Row - Data Quality & Business:**
- **Data Freshness Panel**: Minutes since last update, 24-hour arrival timeline, freshness alerts
- **Data Quality Panel**: Null rate donut chart, schema validation table, completeness trends
- **Business Impact Panel**: Directional accuracy percentage, API usage metrics, ROI calculator

**Additional Features:**
- Alert overlay with popup notifications and history
- Quick links to runbooks and escalation procedures
- Auto-refresh every 30 seconds with manual refresh option

In [1]:
# AAPL Stock Prediction Model - Monitoring Metrics Structure
monitoring = {
    'data': {
        'freshness_minutes': {'threshold': 30, 'alert_level': 'P1'},
        'null_rate_pct': {'threshold': 5, 'alert_level': 'P2'},
        'schema_hash': {'validation': 'exact_match', 'alert_level': 'P1'},
        'price_range_p99': {'threshold': 50, 'alert_level': 'P2'}
    },
    'model': {
        'rolling_7day_mae': {'threshold': 0.05, 'alert_level': 'P1'},
        'population_stability_index': {'threshold': 0.25, 'alert_level': 'P1'},
        'prediction_variance': {'threshold': 0.1, 'alert_level': 'P2'},
        'directional_accuracy_pct': {'threshold': 55, 'alert_level': 'P1'}
    },
    'system': {
        'p95_latency_ms': {'threshold': 200, 'alert_level': 'P0'},
        'error_rate_pct': {'threshold': 1, 'alert_level': 'P0'},
        'cpu_utilization_pct': {'threshold': 80, 'alert_level': 'P1'},
        'requests_per_second': {'threshold': 10, 'alert_level': 'P2'}
    },
    'business': {
        'daily_api_calls': {'threshold': 100, 'alert_level': 'P2'},
        'max_prediction_deviation_pct': {'threshold': 10, 'alert_level': 'P1'},
        'model_roi_ratio': {'threshold': 1.5, 'alert_level': 'P2'}
    }
}

# Display monitoring structure
print("=== AAPL Stock Prediction Model Monitoring ===")
for layer, metrics in monitoring.items():
    print(f"\n{layer.upper()} LAYER:")
    for metric, config in metrics.items():
        threshold_info = f"threshold: {config.get('threshold', 'N/A')}" if 'threshold' in config else f"validation: {config.get('validation', 'N/A')}"
        print(f"  • {metric}: {threshold_info} ({config['alert_level']})")

# Alert escalation summary
print("\n=== ALERT ESCALATION ===")
print("P0: Platform on-call (immediate) → System failures")
print("P1: Data Science team (1 hour) → Model/Data issues") 
print("P2: Product team (24 hours) → Business metric anomalies")

monitoring

=== AAPL Stock Prediction Model Monitoring ===

DATA LAYER:
  • freshness_minutes: threshold: 30 (P1)
  • null_rate_pct: threshold: 5 (P2)
  • schema_hash: validation: exact_match (P1)
  • price_range_p99: threshold: 50 (P2)

MODEL LAYER:
  • rolling_7day_mae: threshold: 0.05 (P1)
  • population_stability_index: threshold: 0.25 (P1)
  • prediction_variance: threshold: 0.1 (P2)
  • directional_accuracy_pct: threshold: 55 (P1)

SYSTEM LAYER:
  • p95_latency_ms: threshold: 200 (P0)
  • error_rate_pct: threshold: 1 (P0)
  • cpu_utilization_pct: threshold: 80 (P1)
  • requests_per_second: threshold: 10 (P2)

BUSINESS LAYER:
  • daily_api_calls: threshold: 100 (P2)
  • max_prediction_deviation_pct: threshold: 10 (P1)
  • model_roi_ratio: threshold: 1.5 (P2)

=== ALERT ESCALATION ===
P0: Platform on-call (immediate) → System failures
P1: Data Science team (1 hour) → Model/Data issues
P2: Product team (24 hours) → Business metric anomalies


{'data': {'freshness_minutes': {'threshold': 30, 'alert_level': 'P1'},
  'null_rate_pct': {'threshold': 5, 'alert_level': 'P2'},
  'schema_hash': {'validation': 'exact_match', 'alert_level': 'P1'},
  'price_range_p99': {'threshold': 50, 'alert_level': 'P2'}},
 'model': {'rolling_7day_mae': {'threshold': 0.05, 'alert_level': 'P1'},
  'population_stability_index': {'threshold': 0.25, 'alert_level': 'P1'},
  'prediction_variance': {'threshold': 0.1, 'alert_level': 'P2'},
  'directional_accuracy_pct': {'threshold': 55, 'alert_level': 'P1'}},
 'system': {'p95_latency_ms': {'threshold': 200, 'alert_level': 'P0'},
  'error_rate_pct': {'threshold': 1, 'alert_level': 'P0'},
  'cpu_utilization_pct': {'threshold': 80, 'alert_level': 'P1'},
  'requests_per_second': {'threshold': 10, 'alert_level': 'P2'}},
 'business': {'daily_api_calls': {'threshold': 100, 'alert_level': 'P2'},
  'max_prediction_deviation_pct': {'threshold': 10, 'alert_level': 'P1'},
  'model_roi_ratio': {'threshold': 1.5, 'alert_